<a href="https://colab.research.google.com/github/JimKing100/DS-Unit-2-Kaggle-Challenge/blob/master/Kaggle_Challenge_Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Installs
%%capture
!pip install --upgrade category_encoders plotly

In [94]:
# Imports
import os, sys

os.chdir('/content')
!git init .
!git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge.git
!git pull origin master

!pip install -r requirements.txt

os.chdir('module1')

Reinitialized existing Git repository in /content/.git/
fatal: remote origin already exists.
From https://github.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge
 * branch            master     -> FETCH_HEAD
Already up to date.


In [0]:
# Disable warning
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
# Imports
import pandas as pd
import numpy as np

import sklearn
sklearn.__version__

# Import the model
from sklearn.linear_model import LogisticRegressionCV

# Import encoder and scaler
import category_encoders as ce
from sklearn.preprocessing import StandardScaler

# Import random forest classifier
from sklearn.ensemble import RandomForestClassifier

In [97]:
# Import, load data and split data into train, validate and test
train_features = pd.read_csv('../data/tanzania/train_features.csv')
train_labels = pd.read_csv('../data/tanzania/train_labels.csv')
test_features = pd.read_csv('../data/tanzania/test_features.csv')
sample_submission = pd.read_csv('../data/tanzania/sample_submission.csv')

assert train_features.shape == (59400, 40)
assert train_labels.shape == (59400, 2)
assert test_features.shape == (14358, 40)
assert sample_submission.shape == (14358, 2)

# Load initial train features and labels
from sklearn.model_selection import train_test_split
X_train = train_features
y_train = train_labels['status_group']

# Split the initial train features and labels 80% into new train and new validation
X_train, X_val, y_train, y_val = train_test_split(
  X_train, y_train, train_size = 0.80, test_size = 0.20,
  stratify = y_train, random_state=42
)

X_train.shape, X_val.shape, y_train.shape, y_val.shape

((47520, 40), (11880, 40), (47520,), (11880,))

In [0]:
# Wrangle train, validate, and test sets
def wrangle(X):
    
    # Set bins value
    bins=20
    
    # Prevent SettingWithCopyWarning
    X = X.copy()
    
    # Clean installer
    X['installer'] = X['installer'].str.lower()
    X['installer'] = X['installer'].str.replace('danid', 'danida')
    X['installer'] = X['installer'].str.replace('disti', 'district council')
    X['installer'] = X['installer'].str.replace('commu', 'community')
    X['installer'] = X['installer'].str.replace('central government', 'government')
    X['installer'] = X['installer'].str.replace('kkkt _ konde and dwe', 'kkkt')
    X['installer'] = X['installer'].str.lower()
    X['installer'] = X['installer'].str[:3]
    X['installer'].value_counts(normalize=True)
    tops = X['installer'].value_counts()[:bins].index
    X.loc[~X['installer'].isin(tops), 'installer'] = 'Other'
    
    # Clean funder and bin
    X['funder'] = X['funder'].str.lower()
    X['funder'] = X['funder'].str[:3]
    X['funder'].value_counts(normalize=True)
    tops = X['funder'].value_counts()[:bins].index
    X.loc[~X['funder'].isin(tops), 'funder'] = 'Other'

    # Use mean for gps_height missing values
    X.loc[X['gps_height'] == 0, 'gps_height'] = X['gps_height'].mean()
    
    # Bin lga
    tops = X['lga'].value_counts()[:bins].index
    X.loc[~X['lga'].isin(tops), 'lga'] = 'Other'

    # Bin ward 
    tops = X['ward'].value_counts()[:bins].index
    X.loc[~X['ward'].isin(tops), 'ward'] = 'Other'
    
    # Bin subvillage
    tops = X_train['subvillage'].value_counts()[:bins].index
    X_train.loc[~X_train['subvillage'].isin(tops), 'subvillage'] = 'Other'

    # Clean latitude and longitude
    average_lat = X.groupby('region').latitude.mean().reset_index()
    average_long = X.groupby('region').longitude.mean().reset_index()

    shinyanga_lat = average_lat.loc[average_lat['region'] == 'Shinyanga', 'latitude']
    shinyanga_long = average_long.loc[average_lat['region'] == 'Shinyanga', 'longitude']

    X.loc[(X['region'] == 'Shinyanga') & (X['latitude'] > -1), ['latitude']] = shinyanga_lat[17]
    X.loc[(X['region'] == 'Shinyanga') & (X['longitude'] == 0), ['longitude']] = shinyanga_long[17]

    mwanza_lat = average_lat.loc[average_lat['region'] == 'Mwanza', 'latitude']
    mwanza_long = average_long.loc[average_lat['region'] == 'Mwanza', 'longitude']

    X.loc[(X['region'] == 'Mwanza') & (X['latitude'] > -1), ['latitude']] = mwanza_lat[13]
    X.loc[(X['region'] == 'Mwanza') & (X['longitude'] == 0) , ['longitude']] = mwanza_long[13]
    
    # Impute mean for tsh based on mean of source_class/basin/waterpoint_type_group
    def tsh_calc(tsh, source, base, waterpoint):
      if tsh == 0:
        if (source, base, waterpoint) in tsh_dict:
          new_tsh = tsh_dict[source, base, waterpoint]
          return new_tsh
      else:
        return tsh
      return tsh
  
    temp = X[X['amount_tsh'] != 0].groupby(['source_class',
                                            'basin',
                                            'waterpoint_type_group'])['amount_tsh'].mean()

    tsh_dict = dict(temp)
    X['amount_tsh'] = X.apply(lambda x: tsh_calc(x['amount_tsh'], x['source_class'], x['basin'], x['waterpoint_type_group']), axis=1)

               
    # quantity & quantity_group are duplicates, so drop quantity_group
    X = X.drop(columns='quantity_group')
    X = X.drop(columns='num_private')
    
    # return the wrangled dataframe
    return X




In [99]:
# Wrangle the data
X_train = wrangle(X_train)
X_val = wrangle(X_val)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [0]:
# Feature engineering
def feature_engineer(X):
  
  # Create new feature pump_age
  X['pump_age'] = 2013 - X['construction_year']
  X.loc[X['pump_age'] == 2013, 'pump_age'] = 0
  X.loc[X['pump_age'] == 0, 'pump_age'] = 10
  
  # Create new feature region_district
  X['region_district'] = X['region_code'].astype(str) + X['district_code'].astype(str)

  return X

In [0]:
# Feature engineer the data
X_train = feature_engineer(X_train)
X_val = feature_engineer(X_val)

In [0]:
# Encode a feature
def encode_feature(X, y, str):
  X['status_group'] = y
  X.groupby(str)['status_group'].value_counts(normalize=True)
  X['functional']= (X['status_group'] == 'functional').astype(int)
  X[['status_group', 'functional']]
  return X

In [0]:
# Encode all the categorical features
train = X_train.copy()
train = encode_feature(train, y_train, 'quantity')
train = X_train.copy()
train = encode_feature(train, y_train, 'waterpoint_type')
train = X_train.copy()
train = encode_feature(train, y_train, 'extraction_type')
train = X_train.copy()
train = encode_feature(train, y_train, 'installer')
train = X_train.copy()
train = encode_feature(train, y_train, 'basin')
train = X_train.copy()
train = encode_feature(train, y_train, 'region')
train = X_train.copy()
train = encode_feature(train, y_train, 'payment')
train = X_train.copy()
train = encode_feature(train, y_train, 'source')
train = X_train.copy()
train = encode_feature(train, y_train, 'lga')
train = X_train.copy()
train = encode_feature(train, y_train, 'scheme_management')
train = X_train.copy()
train = encode_feature(train, y_train, 'management')
train = X_train.copy()
train = encode_feature(train, y_train, 'region_code')
train = X_train.copy()
train = encode_feature(train, y_train, 'subvillage')
train = X_train.copy()
train = encode_feature(train, y_train, 'funder')
train = X_train.copy()
train = encode_feature(train, y_train, 'water_quality')
train = X_train.copy()
train = encode_feature(train, y_train, 'ward')

In [104]:
# use quantity feature and the numerical features but drop id
categorical_features = ['quantity', 'waterpoint_type', 'extraction_type', 'installer',
                         'basin', 'region', 'payment', 'source', 'lga', 'public_meeting',
                         'scheme_management', 'permit', 'management', 'region_district',
                         'subvillage', 'funder', 'water_quality', 'ward']
                                              
numeric_features = X_train.select_dtypes('number').columns.drop('id').tolist()
features = categorical_features + numeric_features

# make subsets using the quantity feature all numeric features except id
X_train_subset = X_train[features]
X_val_subset = X_val[features]

# Do the encoding
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train_encoded = encoder.fit_transform(X_train_subset)
X_val_encoded = encoder.transform(X_val_subset)

# Use the scaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_val_scaled = scaler.transform(X_val_encoded)

# Fit the model and check the accuracy
model = LogisticRegressionCV(n_jobs = -1)
model.fit(X_train_scaled, y_train)

print('Validation Accuracy', model.score(X_val_scaled, y_val));

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Validation Accuracy 0.7515151515151515


In [105]:
model = RandomForestClassifier(n_estimators=1000, 
                               random_state=42,
                               max_features = 'auto',
                               n_jobs=-1,
                               verbose = 1)

model.fit(X_train_scaled, y_train)

print('Validation Accuracy', model.score(X_val_scaled, y_val));

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   12.5s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   28.7s
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:   51.4s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:  1.1min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.7s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:    1.2s


Validation Accuracy 0.8106060606060606


[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed:    1.5s finished
